# SRCNN Using Theano

This is a notebook on theano based Image-super resolution based on this [paper](https://arxiv.org/abs/1501.00092).
This implementation is based on the implementation of ['corochann'](https://github.com/corochann).
The original codes of this implementation can be found [here](https://github.com/corochann/theanonSR) 

## Introduction to Image Super_resolution

Single image super-resolution (SR), which aims at recovering a high-resolution image from a single low-resolution image, is a classical problem in computer vision. This problem is inherently ill-posed since a multiplicity of solutions exist for any given low-resolution pixel.

Such a problem is typically mitigated by constraining the solution space by strong prior information. To learn the prior,recent state-of-the-art methods mostly adopt example-based strategy. These methods either exploit internal similarities of the same image or learn mappingfunctions from external low- and high-resolution exemplar pairs.

In this paper a Deep Convolutional Neural Network has been developed to solve this problem.The proposed Super-Resolution-convoluted-neural-network, SRCNN, has several appealing properties. First, its structure is intentionally designed with simplicity in mind, and yet provides superior accuracy.Secondly, with morderate numbers of filters and layers, this method achieves fast speed for practical on-line usage even on a CPU.

## The model

Consider a single low-resolution image, we first upscale it to the desired size using bicubic interpolation, which is the only pre-processing we perform. Let us denote the interpolated image as $Y$. Our goal is to recover from $Y$ an image $F(Y)$ that is as similaras possible to the ground truth high-resolution image $X$. For the ease of presentation, we still call $Y$ a “low-resolution” image, although it has the same size as $X$. We wish to learn a mapping $F$, which conceptually consists of three operations: 

1) <b>Patch extraction and representation:</b> this operation extracts (overlapping) patches from the low-resolution image $Y$ and represents each patch as a high-dimensional vector. These vectors comprise a set of feature maps, of which the number equals to the dimensionality of the vectors. 

2) <b>Non-linear mapping:</b> this operation nonlinearly maps each high-dimensional vector onto another high-dimensional vector. Each mapped vector is conceptually the representation of a high-resolution patch. These vectors comprise another set of feature maps. 

3) <b>Reconstruction:</b> this operation aggregates the above high-resolution patch-wise representations to generate the final high-resolution image. This image is expected to be similar to the ground truth $X$. 



![model image](model.png)


This can be mathematically be represented as

1) 
\begin{equation}
F_1 (Y) = max(0,W_1*Y +B_1)
\end{equation}
where $W_1$ and $B_1$ represent the filters and biases respectively, and ’*’ denotes the convolution operation. Here,$ W_1$ corresponds to $n_1$ filters of support $c\times f_1 \times f_1$, where $c$ is the number of channels in the input image, $f_1$ is the spatial size of a filter.

2) 
\begin{equation}
F_2 (Y) = max(0,W_2*F_1(Y) +B_2)
\end{equation}
Here, $W_2$ contains $n_2$ filters of size $n_1\times f_2 \times f_2$, and $B_2$ is $n_2$-dimensional.

3) 
\begin{equation}
F(Y) = W_3*F_2(Y) +B_3
\end{equation}
Here $W_3$ corresponds to $c$ filters of a size $n_2 \times f_3\times f_3$, and $B_3$ is a $c$-dimensional vector.

## Training

Learning the end-to-end mapping function $F$ requires the estimation of network parameters $\Theta = \{ W_1;W_2;W_3;B_1;B_2;B_3\} $. This is achieved through min-imizing the loss between the reconstructed images $F(Y; \Theta)$ and the corresponding ground truth high-resolution images $X$. Given a set of high-resolution images $\{Xi\}$ and their corresponding low-resolution images $\{Yi\}$, we use Mean Squared Error (MSE) as the loss function: 

\begin{equation}
L(\Theta) = \frac{1}{n} \sum_{i=1}^{n}{{|| F(Y_i,\Theta) - X_i ||}^2},
\end{equation}
where $n$ is the number of training samples

# Main code

1) Load the image

2) preprocess the image

3) Load the pretrained model

4) Get output from the pretrained model

5) Rescale it and ave it with the stats

In [1]:
# Main Packages and Libraries
from __future__ import print_function
import argparse


try:
    import cPickle as pickle
except:
    import pickle as pickle

import os
import cv2
import json

import numpy as np
import theano
import theano.tensor as T

#from layer import ConvLayer
#from tools.image_processing import preprocess

ImportError: No module named layer

In [ ]:
## The main Function Loop

#Loading the image
input_img = cv2.imread(photo_file_path, cv2.IMREAD_COLOR)
input_image_height = input_img.shape[0]
input_image_width = input_img.shape[1]

# Output image size
output_image_height = 2 * input_image_height
output_image_width = 2 * input_image_width

# Load the trained model
f = open(os.path.join(model_folder, 'train.json'), 'r')
model_data = json.load(f)

# model parameters
rng = np.random.RandomState(model_data["rng_seed"])
input_channel_number = model_data["input_channel"]
layer_count = model_data["layer_count"]
#batch_size = model_data["minibatch_size"]
batch_size = 1  # We will convert one photo file
layers = model_data["layers"]

# WE ARE TAKING SYMMETRICAL FILTERS ONLY
# And ofcourse odd number as the filter length
image_padding = 0
for i in np.arange(layer_count):
    image_padding += layers[i]["filter_height"] - 1
total_image_padding = image_padding
## Why is this being done?

## Making the theano variables
index = T.lscalar()  # index to a minibatch
x = T.tensor4(name='x')  # input data (rasterized images): (batch_size, ch, image_height, image_width)
y = T.tensor4(name='y')  # output data (rasterized images): (batch_size, ch, image_height, image_width)
layer0_input = x # x.reshape((batch_size, input_channel_number, output_image_height, output_image_width))
# shape(batch_size, #of feature map, image height, image width)

In [ ]:
# Loading the best trained model from pickled file
# Refer Conv layer

param_lists = pickle.load(open(os.path.join(model_folder, 'best_model.pkl')))
conv_layers = []
for i in np.arange(layer_count):
    if i == 0:
        previous_layer_channel = input_channel_number
        layer_input = layer0_input
    else:
        previous_layer_channel = layers[i-1]["channel"]
        layer_input = conv_layers[-1].output
    layer = ConvLayer(
        rng,
        input=layer_input,
        image_shape=(batch_size, previous_layer_channel, output_image_height + image_padding, output_image_width + image_padding),
        filter_shape=(layers[i]["channel"], previous_layer_channel, layers[i]["filter_height"], layers[i]["filter_width"]),
        W_values=param_lists[i][0],
        b_values=param_lists[i][1]
    )
    conv_layers.append(layer)
    
cost = conv_layers[-1].cost(y)

In [ ]:
# Converting the read image into YCC format
ycc_input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2YCR_CB)
data_prescaled_x = np.empty((1, input_channel_number, input_image_height, input_image_width))
data_prescaled_x[0, :, :, :] = np.transpose(ycc_input_img[:, :, 0:1], (2, 0, 1))
## REFER PREPROCESSING
data_scaled_x = preprocess(data_prescaled_x, total_image_padding // 2)
input_x = theano.shared(np.asarray(data_scaled_x, dtype=theano.config.floatX), borrow=True)

In [ ]:
## Predicting
srcnn_photo_predict = theano.function([],conv_layers[-1].output,
                                      givens={x: input_x[:]})
output_img_y = srcnn_photo_predict()
img0 = output_img_y[0].transpose(1, 2, 0) * 256.
scaled_input_img = cv2.resize(input_img, (output_image_width, output_image_height))

if compare:
    cv2.imwrite(conventional_file_path, scaled_input_img)

ycc_scaled_input_img = cv2.cvtColor(scaled_input_img, cv2.COLOR_BGR2YCR_CB)
ycc_scaled_input_img[:, :, 0:1] = img0  # (width, height, ch)
rgb_scaled_img = cv2.cvtColor(ycc_scaled_input_img, cv2.COLOR_YCR_CB2BGR)
cv2.imwrite(output_file_path, rgb_scaled_img)

# Layer Code

1) Get the size of the conv layer, filter size, number etc

2) make loss function

In [ ]:
class ConvLayer(object):
    
    def __init__(self, rng, input, filter_shape, image_shape, activation=None,
             W_values=None, b_values=None,
             use_adam=False):
        """
        :param rng: RandomState - random number generator
        :param input: dtensor4 (batch_size, #of input feature map, image height, image width)
        :param filter_shape: (W is filter) tuple/list of length 4
        (#of filters (output feature map), #of input feature map, filter height, filter width)
        :param image_shape: tuple/list of length 4 (batch_size, #of input feature map, image height, image width)
        :param activation: Not used for now, instead we use LReLU as default
        :param W_values: None for initialize with random value (Training phase)
                  set numpy.ndarray for specifying W_value (Trained value will be set here in Application phase)
                  4D array - (#of filters (output feature map), #of input feature map, filter height, filter width)
                  [NOTE] self.W is theano.tensor.shared.var.TensorSharedVariable, but this W is numpy.adarray (only value)
        :param b_values: None for initialize with random value (Training phase)
                  set numpy.ndarray for specifying W_value (Trained value will be set here in Application phase)
                  [NOTE] self.b is theano.tensor.shared.var.TensorSharedVariable, but this b is numpy.adarray (only value)
                  1D array - (#of filters (output feature map), )
        :param use_adam: deprecated, should be always True. (True when use ADAM)
        :return:
        """
        assert image_shape[1] == filter_shape[1]  # make sure they are the same
        # pad outside, to maintain input image & output image's height/width same.
        self.input = input

        #fan_in - #of input to convolution layer = #of input feature maps * filter height * filter width"
        fan_in = np.prod(filter_shape[1:])
        #fan_out - #of output which one element affects to = #of output * filter height * filter width"
        fan_out = (filter_shape[0]) * np.prod(filter_shape[2:])

        # init @: 4d tensor(filter_shape)
        if W_values is None:
            # Training phase, init with random value
            W_bound = np.sqrt(6. / (fan_in + fan_out))
            # W_bound = 1. / fan_in
            W_values = np.asarray(
                rng.uniform(low=-1.0*W_bound, high=1.2*W_bound, size=filter_shape),
                dtype=theano.config.floatX
            )

        self.W = theano.shared(W_values, borrow=True)
        # for the bias
        if b_values is None:
            # Training phase, init with 0
            b_values = np.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        if use_adam:
            zero_W_values = np.zeros(filter_shape, dtype=theano.config.floatX)
            self.W_m = theano.shared(value=zero_W_values, borrow=False)
            self.W_v = theano.shared(value=zero_W_values, borrow=False)
            self.b_m = theano.shared(value=b_values, borrow=False)
            self.b_v = theano.shared(value=b_values, borrow=False)  # for momentum
        # CONVOLUTION
        # image_shape: tuple/list of length 4 (batch_size, #of input feature map, image height, image width)
        #conv_out: 4d tensor (batch_size, #of output feature map, output height, output width)
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            border_mode='valid',
            input_shape=image_shape
        )
        conv_out_plus_b = conv_out + self.b.dimshuffle('x', 0, 'x', 'x')
        # TODO: consider activation
        # - sigmoid, tanh, LeRU (max(0, x)) etc.

        # - relu: vanishing gradient occurs
        # self.output = T.minimum(T.maximum(0, conv_out + self.b.dimshuffle('x', 0, 'x', 'x')), 1)

        # - lrelu: leak rectified linear unit
        # self.output = T.switch(T.lt(conv_out_plus_b, 0), -0.1*conv_out_plus_b, conv_out_plus_b) # LReLU

        # - cropped leaky relu -> cropped to 0-1
        self.output = T.switch(T.lt(conv_out_plus_b, - 1179 / 256.), 0,
                               T.switch(T.lt(conv_out_plus_b, 1 / 256.), conv_out_plus_b / 1280. + 1279 / 327680.,
                                        T.switch(T.lt(conv_out_plus_b, 255 / 256.), conv_out_plus_b,
                                                 T.switch(T.lt(conv_out_plus_b, 1535 / 256.), conv_out_plus_b / 1280. + 326145. / 327680.,
                                                          1))))
        self.params = [self.W, self.b]

        if use_adam:
            self.params_m = [self.W_m, self.b_m]
            self.params_v = [self.W_v, self.b_v]

    def cost(self, y):
        """ calculating cost by Mean Squared Error (MSE) as the loss function
        NOTE: y and self.output must be same tensor size
        :param y: 4d tensor (batch_size, #of feature map (3 for RGB), output height, output width)
        :return:
        """
        return T.mean(T.sqr(y - self.output))

# Image preprocessing

In [ ]:
def preprocess(pre_scaled_x, image_padding=0):
    """
    preprocessing image consists of 3 parts
    1. Scaling: scale original input image to twice scale, using nearest neighbor method.
    2. Normalization: normalize each pixel's value from 0~256 to 0~1
    3. Padding: pad edge using np.pad.
                because image size will reduce during convolution in neural network
    :param pre_scaled_x:  original input image (numpy.array)
    :param image_padding: value of pixel to be padded
    :return:
    """
    print('preprocess...')
    print('pre_scaled_x.shape', pre_scaled_x.shape)
    scaled_x = np.empty((pre_scaled_x.shape[0],
                         pre_scaled_x.shape[1],
                         pre_scaled_x.shape[2] * 2,
                         pre_scaled_x.shape[3] * 2),
                        )
    # dtype=train_set_x.dtype)
    for k in np.arange(pre_scaled_x.shape[2]):
        for l in np.arange(pre_scaled_x.shape[3]):
            # scaled_x[i][j, 2 * k: 2 * k + 1, 2 * l: 2 * l + 1] = pre_scaled_x[i, j, k, l] / 256.
            scaled_x[:, :, 2 * k, 2 * l] = pre_scaled_x[:, :, k, l] / 256.
            scaled_x[:, :, 2 * k, 2 * l + 1] = pre_scaled_x[:, :, k, l] / 256.
            scaled_x[:, :, 2 * k + 1, 2 * l] = pre_scaled_x[:, :, k, l] / 256.
            scaled_x[:, :, 2 * k + 1, 2 * l + 1] = pre_scaled_x[:, :, k, l] / 256.

    # print('pre_scaled_x = ', pre_scaled_x)
    # print('scaled_x = ', scaled_x)

    # PADDING
    if image_padding > 0:
        print('image padding ', image_padding, ' pixels...')
        new_img = np.empty((scaled_x.shape[0], scaled_x.shape[1],
                            scaled_x.shape[2] + 2 * image_padding,
                            scaled_x.shape[3] + 2 * image_padding), dtype=scaled_x.dtype)
        for i in np.arange(scaled_x.shape[0]):
            for j in np.arange(scaled_x.shape[1]):
                new_img[i, j, :, :] = np.pad(scaled_x[i, j, :, :], image_padding, "edge")
        scaled_x = new_img

    return scaled_x


# Data preparation
To build Low res files and high res files from given images.

In [ ]:
def build_data(image_save_flag=False):
    index = 0
    logfile = open(logfile_name, 'w')

    if image_save_flag:
        if not os.path.exists(cropped_directory):
            os.makedirs(cropped_directory)
        if not os.path.exists(half_directory):
            os.makedirs(half_directory)

    for root, dirs, files in os.walk(input_directory):
        # print root, dirs, files
        batch_size = len(files)
        print('file size', len(files))
        data_x = np.empty((batch_size, image_channel_number, crop_height / 2, crop_width / 2))
        data_y = np.empty((batch_size, image_channel_number, crop_height, crop_width))

        for file in files:
            img = cv2.imread(os.path.join(root, file))

            # crop largest square image from center
            height = img.shape[0]
            width = img.shape[1]
            shorter_edge = min(height, width)
            if shorter_edge < crop_height:
                print('skip', file)
                print('skip', file, file=logfile)
                continue
            # print 'shorter_edge', shorter_edge  # (h, w, 3) = (height, width, channel RGB)  where h == w
            # NOTE: its img[y: y + h, x: x + w] and *not* img[x: x + w, y: y + h]
            square_img = img[
                       height // 2 - shorter_edge // 2: height // 2 + shorter_edge // 2,
                       width // 2 - shorter_edge // 2: width // 2 + shorter_edge // 2,
                       :]

            # print square_img.shape  # (h, w, 3) = (height, width, channel RGB)  where h == w
            # crop_img = cv2.resize(square_img, (crop_height, crop_width))
            crop_img = img[
                       height // 2 - crop_height // 2: height // 2 + crop_height // 2,
                       width // 2 - crop_width // 2: width // 2 + crop_width // 2,
                       :]

            # print crop_img.shape  # (h, w, 3) = (height, width, channel RGB)  where h == w = 256
            # save y image
            if image_save_flag:
                print('saving to ', os.path.join(cropped_directory, file))
                cv2.imwrite(os.path.join(cropped_directory, file), crop_img)

            # Construct answer data y
            # convert from RGB to YCbCr
            ycc_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2YCR_CB)
            transposed_y_img = np.transpose(crop_img[:, :, 0:1], (2, 0, 1))  # (ch, height, width)

            data_y[index, :, :, :] = transposed_y_img
            # print transposed_y_img.shape  # (3, 420, 640) = (channel, height, width)

            # resize image half. NOTE: size order is (width, height)
            half_img = cv2.resize(ycc_img, (crop_width // 2, crop_height // 2))
            # print half_img.shape
            if image_save_flag:
                print('saving to ', os.path.join(half_directory, file))
                cv2.imwrite(os.path.join(half_directory, file), half_img)

            ycc_half_img = cv2.cvtColor(half_img, cv2.COLOR_BGR2YCR_CB)
            transposed_y_half_img = np.transpose(ycc_half_img[:, :, 0:1], (2, 0, 1))  # (ch, height, width)
            data_x[index] = transposed_y_half_img
            # cv2.imwrite(os.path.join(half_directory, file), half_img)
            index += 1

        # print 'data_x.shape', data_x.shape
        # print 'data_y.shape', data_y.shape
        # dataset = (shared_x, shared_y)
        dataset = (data_x, data_y)

        return dataset



# Train SRCNN


In [ ]:
def execute_srcnn(n_epochs=1000,
                  image_height=232,
                  image_width=232,
                  resume=False):
    """
    :param n_epochs:
    :param batch_size: minibatch_size to train
    :param image_height:
    :param image_width:
    :return:
    """
    train_log_file = open(os.path.join(training_model_folder, train_log_file_name), 'w')

    # model data loading
    f = open(os.path.join(training_model_folder, 'train.json'))
    model_data = json.load(f)
    rng = np.random.RandomState(model_data["rng_seed"])

    # #of feature map = 1 - Y only from YCbCr, you need to modify a bit to support 3 - RGB channel
    input_channel_number = model_data["input_channel"]
    layer_count = model_data["layer_count"]
    batch_size = model_data["minibatch_size"]
    layers = model_data["layers"]

    image_padding = 0
    for i in np.arange(layer_count):
        # currently it only supports filter height == filter width
        assert layers[i]["filter_height"] == layers[i]["filter_width"], "filter height and filter width must be same!"
        assert layers[i]["filter_height"] % 2 == 1, "Only odd number filter is supported!"
        image_padding += layers[i]["filter_height"] - 1

    total_image_padding = image_padding

    # training data loading
    datasets = load_data()

    np_train_dataset, np_valid_dataset, np_test_dataset = datasets
    np_train_set_x, np_train_set_y = np_train_dataset
    np_valid_set_x, np_valid_set_y = np_valid_dataset
    np_test_set_x, np_test_set_y = np_test_dataset

    # print('train_set_x.shape[0]', train_set_x.shape[0].eval())
    # PREPROCESSING
    start_time = timeit.default_timer()
    train_scaled_x = preprocess(np_train_set_x, total_image_padding // 2)
    valid_scaled_x = preprocess(np_valid_set_x, total_image_padding // 2)
    test_scaled_x = preprocess(np_test_set_x, total_image_padding // 2)
    end_time = timeit.default_timer()
    print('preprocess time %i sec' % (end_time - start_time))
    print('preprocess time %i sec' % (end_time - start_time), file=train_log_file)

    # print('scaled_x', scaled_x)

    def shared_dataset(data, borrow=True):
        shared_data = theano.shared(np.asarray(data, dtype=theano.config.floatX), borrow=borrow)
        return shared_data

    train_set_x = shared_dataset(train_scaled_x)
    valid_set_x = shared_dataset(valid_scaled_x)
    test_set_x = shared_dataset(test_scaled_x)
    train_set_y = shared_dataset(np_train_set_y / 256.) # normalize
    valid_set_y = shared_dataset(np_valid_set_y / 256.)
    test_set_y = shared_dataset(np_test_set_y / 256.)

    train_set_batch_size = np_train_set_x.shape[0]
    n_train_batches = np_train_set_x.shape[0] // batch_size
    n_valid_batches = np_valid_set_x.shape[0] // batch_size
    n_test_batches = np_test_set_x.shape[0] // batch_size

    # SHOW Test images (0~5)
    test_img_batch = test_set_x.get_value(borrow=False)[0: 5]  # OK.
    for i in np.arange(5):
        cv2.imwrite(os.path.join(training_process_folder, 'photo' + str(i) + '_input.jpg'),
                    test_img_batch[i].transpose(1, 2, 0) * 256.)

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a minibatch
    indexes = T.lvector()  # index randomizer
    x = T.tensor4(name='x')  # input data (rasterized images): (batch_size, ch, image_height, image_width)
    y = T.tensor4(name='y')  # output data (rasterized images): (batch_size, ch, image_height, image_width)

    # BUILD MODEL
    print('... building the model')
    if resume:
        param_lists = pickle.load(open(os.path.join(training_model_folder, 'best_model.pkl')))
    layer0_input = x.reshape((batch_size,
                              input_channel_number,
                              image_height + total_image_padding,
                              image_width + total_image_padding))  # shape(batch_size, #of feature map, image height, image width)

    ConvLayers = []
    updates = []

    for i in np.arange(layer_count):
        if i == 0:
            previous_layer_channel = input_channel_number
            layer_input = layer0_input
        else:
            previous_layer_channel = layers[i-1]["channel"]
            layer_input = ConvLayers[-1].output

        #print('[DEBUG], i = %i, layers[i]["channel"] = %i, layers[i]["filter_height"] = %i, layers[i]["filter_width"] = %i' %
        #      (i, layers[i]["channel"], layers[i]["filter_height"], layers[i]["filter_width"]))
        if resume:
            layer = ConvLayer(
                rng,
                input=layer_input,
                image_shape=(batch_size, previous_layer_channel, image_height + image_padding, image_width + image_padding),
                filter_shape=(layers[i]["channel"], previous_layer_channel, layers[i]["filter_height"], layers[i]["filter_width"]),
                use_adam=True,
                W_values=param_lists[i][0],
                b_values=param_lists[i][1]
            )
        else:
            layer = ConvLayer(
                rng,
                input=layer_input,
                image_shape=(batch_size, previous_layer_channel, image_height + image_padding, image_width + image_padding),
                filter_shape=(layers[i]["channel"], previous_layer_channel, layers[i]["filter_height"], layers[i]["filter_width"]),
                use_adam=True
            )
        ConvLayers.append(layer)
        image_padding -= (layers[i]["filter_height"] - 1)
        # print('[DEBUG] image_padding = ', image_padding)

    # crete train model
    # alpha = 0.001
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 0.1  # 0.000001
    beta1_t = theano.shared(value=np.cast['float32'](0.0), borrow=True)  # Casting float32 is necessary for GPU usage
    beta2_t = theano.shared(value=np.cast['float32'](.9), borrow=True)

    cost = ConvLayers[-1].cost(y)

    updates.append((beta1_t, beta1_t * beta1))
    updates.append((beta2_t, beta1_t * beta2))
    for i in np.arange(layer_count):
        params = ConvLayers[i].params
        params_m = ConvLayers[i].params_m # for adam
        params_v = ConvLayers[i].params_v # for adam
        gparams = T.grad(cost, params)
        for param, gparam, param_m, param_v in zip(params, gparams, params_m, params_v):
            # Adam
            updates.append((param_m, beta1 * param_m + (1 - beta1) * gparam))
            updates.append((param_v, beta2 * param_v + (1 - beta2) * gparam * gparam))
            updates.append((param, param - layers[i]["learning_rate"] * param_m / (1. - beta1_t) / (T.sqrt(param_v / (1 - beta2_t)) + epsilon)))
            # Normal SGD
            # updates.append((param, param - layers[i]["learning_rate"] * gparam))

    # indexes is used for image sequence randomizer for training
    train_model = theano.function(
        [index, indexes],
        cost,
        updates=updates,
        givens={
            x: train_set_x[indexes[index * batch_size: (index + 1) * batch_size]],
            y: train_set_y[indexes[index * batch_size: (index + 1) * batch_size]]
        }
    )
    # create a test function
    # TODO: implement error function (test evaluation function, e.g. PSNR), instead of using cost function
    test_model = theano.function(
        [index],
        ConvLayers[-1].cost(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        [index],
        ConvLayers[-1].cost(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    construct_photo = theano.function(
        [index],
        ConvLayers[-1].output,
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size]
        }
    )

    # TRAIN MODEL
    print('... training')
    patience = 30000  # 10000
    patience_increase = 2
    improvement_threshold = 0.998  # 0.995

    validation_frequency = min(n_train_batches, patience // 2) * 2

    best_validation_loss = np.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False

    while (epoch < n_epochs) and (not done_looping):
        start_time_for_each_epoch = timeit.default_timer()
        epoch += 1
        mean_cost = []
        random_indexes = np.random.permutation(train_set_batch_size)  # index randomizer
        for minibatch_index in range(n_train_batches):
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if iter < 10:
                img_batch = construct_photo(0)
                img0 = img_batch[0].transpose(1, 2, 0) * 256.
                print('[DEBUG] iter ', iter, ' img0: ', img0)

            if iter % 100 == 0:
                print('training @ iter ', iter)
            mean_cost += [train_model(minibatch_index, random_indexes)]

            if (iter + 1) % validation_frequency == 0:
                validation_losses = [validate_model(i) for i in range(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)
                print('epoch %i, minibatch %i/%i, validation cost %f' %
                      (epoch, minibatch_index + 1, n_train_batches, this_validation_loss))
                print('epoch %i, minibatch %i/%i, validation cost %f' %
                      (epoch, minibatch_index + 1, n_train_batches, this_validation_loss),
                      file=train_log_file)

                if this_validation_loss < best_validation_loss:
                    if this_validation_loss < best_validation_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                        print('update patience -> ', patience, ' iter')
                        # we will execute at least patience iter.

                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    test_losses = [test_model(i) for i in range(n_test_batches)]
                    test_score = np.mean(test_losses)
                    print('     epoch %i, minibatch %i/%i, test cost of best model %f' %
                          (epoch, minibatch_index + 1, n_train_batches, test_score))
                    print('     epoch %i, minibatch %i/%i, test cost of best model %f' %
                          (epoch, minibatch_index + 1, n_train_batches, test_score),
                          file=train_log_file)
                    # Save best model
                    with open(os.path.join(training_model_folder, 'best_model.pkl'), 'wb') as f:
                        param_lists = []
                        for i in np.arange(layer_count):
                            param_lists.append([ConvLayers[i].W.get_value(), ConvLayers[i].b.get_value()])
                        pickle.dump(param_lists, f)

            if patience <= iter:
                done_looping = True
                break
        end_time_for_each_epoch = timeit.default_timer()
        diff_time = end_time_for_each_epoch - start_time_for_each_epoch
        print('Training epoch %d cost is %f, took %i min %i sec' %
              (epoch, np.mean(mean_cost), diff_time / 60., diff_time % 60))
        print('Training epoch %d cost is %f, took %i min %i sec' %
              (epoch, np.mean(mean_cost), diff_time / 60., diff_time % 60),
              file=train_log_file)

        # for checking/monitoring the training process
        if epoch // 10 == 0 or epoch % 10 == 0:
            photo_num = 0
            loop = 0
            while photo_num < 5:
                img_batch = construct_photo(loop)
                for j in np.arange(batch_size):
                    if photo_num == 0:
                        print('output_img0: ', img_batch[j].transpose(1, 2, 0) * 256.)
                    cv2.imwrite(os.path.join(training_process_folder,
                                             'photo' + str(photo_num) + '_epoch' + str(epoch) + '.jpg'),
                                img_batch[j].transpose(1, 2, 0) * 256.)
                    photo_num += 1
                    if photo_num == 5:
                        break
                loop += 1

    end_time = timeit.default_timer()

    print('Optimization complete.')
    print('Best validation score of %f %% obtained at iteration %i, '
          'with test performance %f' %
          (best_validation_loss, best_iter + 1, test_score))
    print(('The code for file ' +
           os.path.split(__file__)[1] +
           ' ran for %.2fm' % ((end_time - start_time) / 60.)), file=sys.stderr)

    print('Optimization complete.',
          file=train_log_file)
    print('Best validation score of %f obtained at iteration %i, '
          'with test performance %f' %
          (best_validation_loss, best_iter + 1, test_score),
          file=train_log_file)
    print(('The code for file ' +
           os.path.split(__file__)[1] +
           ' ran for %.2fm' % ((end_time - start_time) / 60.)),
          file=train_log_file)


In [ ]:

def predict_test_set(image_height=232,
                     image_width=232,
                     model_folder=training_model_folder):
    print('predict...')
    if not os.path.exists(model_folder):
        print('os.getcwd() ', os.getcwd())
        print(model_folder + ' does not exist!')
        return

    training_process_folder = os.path.join(model_folder, 'training_process')
    f = open(os.path.join(model_folder, 'train.json'), 'r')
    model_data = json.load(f)

    rng = np.random.RandomState(model_data["rng_seed"])
    input_channel_number = model_data["input_channel"]
    layer_count = model_data["layer_count"]
    batch_size = model_data["minibatch_size"]
    layers = model_data["layers"]

    image_padding = 0
    for i in np.arange(layer_count):
        # currently it only supports filter height == filter width
        assert layers[i]["filter_height"] == layers[i]["filter_width"], "filter height and filter width must be same!"
        assert layers[i]["filter_height"] % 2 == 1, "Only odd number filter is supported!"
        image_padding += layers[i]["filter_height"] - 1

    total_image_padding = image_padding

    index = T.lscalar()  # index to a minibatch
    x = T.tensor4(name='x')  # input data (rasterized images): (batch_size, ch, image_height, image_width)
    y = T.tensor4(name='y')  # output data (rasterized images): (batch_size, ch, image_height, image_width)
    layer0_input = x # x.reshape((batch_size, input_channel_number, image_height, image_width))  # shape(batch_size, #of feature map, image height, image width)

    param_lists = pickle.load(open(os.path.join(model_folder, 'best_model.pkl')))
    #print('param_lists', param_lists)
    #print('param_lists1', param_lists[1])
    ConvLayers = []

    for i in np.arange(layer_count):
        if i == 0:
            previous_layer_channel = input_channel_number
            layer_input = layer0_input
        else:
            previous_layer_channel = layers[i-1]["channel"]
            layer_input = ConvLayers[-1].output

        layer = ConvLayer(
            rng,
            input=layer_input,
            image_shape=(batch_size, previous_layer_channel, image_height + image_padding, image_width + image_padding),
            filter_shape=(layers[i]["channel"], previous_layer_channel, layers[i]["filter_height"], layers[i]["filter_width"]),
            W_values=param_lists[i][0],
            b_values=param_lists[i][1]
        )
        #print('layer.W', layer.W, layer.W.get_value())
        #print('layer.b', layer.b, layer.b.get_value())
        ConvLayers.append(layer)

    #print('ConvLayers', ConvLayers)
    # crete train model
    cost = ConvLayers[-1].cost(y)

    datasets = load_data()
    train_dataset, valid_dataset, test_dataset = datasets
    # train_set_x, train_set_y = train_dataset
    # valid_set_x, valid_set_y = valid_dataset
    np_test_set_x, np_test_set_y = test_dataset

    # PREPROCESSING
    test_scaled_x = preprocess(np_test_set_x, total_image_padding // 2)
    test_set_x = theano.shared(np.asarray(test_scaled_x, dtype=theano.config.floatX), borrow=True)
    #print('test_scaled_x', test_scaled_x)

    construct_photo_predict = theano.function(
        [index],
        ConvLayers[-1].output,
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size]
        }
    )

    photo_num = 0
    loop = 0
    while photo_num < 5:
        img_batch = construct_photo_predict(loop)
        for j in np.arange(batch_size):
            if photo_num == 0:
                print('output_img0: ', img_batch[j].transpose(1, 2, 0) * 256.)
            cv2.imwrite(os.path.join(training_process_folder,
                                     'photo' + str(photo_num) + '_predict.jpg'),
                        img_batch[j].transpose(1, 2, 0) * 256.)
            photo_num += 1
            if photo_num == 5:
                break
        loop += 1